# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas as hvplot
import pandas as pd
import requests
from pathlib import Path


# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,sabang,5.8933,95.3214,29.77,68,50,3.45,ID,1713786578
1,1,olonkinbyen,70.9221,-8.7187,-3.32,66,0,7.34,SJ,1713786579
2,2,albany,42.6001,-73.9662,1.86,57,6,3.26,US,1713786377
3,3,hermanus,-34.4187,19.2345,20.20,70,37,3.94,ZA,1713786581
4,4,port mathurin,-19.6833,63.4167,27.18,78,31,6.91,MU,1713786581


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat',  # Specify longitude and latitude columns
    size='Humidity',  # Use humidity for point size
    geo=True,  # Use geographic coordinates
    tiles='OSM',  # Choose a map tile provider
    hover_cols=['City'],  # Specify columns for hover information
    color="City"
)
# Display the map plot
map_plot


C:\Users\sri_p\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# Define ideal weather conditions
ideal_conditions = {
    "Max Temp": [21, 27],  # Temperature between 21°C and 27°C
    "Wind Speed": 4.5,     # Wind speed less than 4.5 m/s
    "Cloudiness": 0        # Cloudiness equal to 0
}

# Filter the DataFrame based on ideal weather conditions
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] > ideal_conditions['Max Temp'][0]) & 
    (city_data_df['Max Temp'] < ideal_conditions['Max Temp'][1]) & 
    (city_data_df['Wind Speed'] < ideal_conditions['Wind Speed']) & 
    (city_data_df['Cloudiness'] == ideal_conditions['Cloudiness'])
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
148,148,ilhabela,-23.7781,-45.3581,24.60,67,0,0.10,BR,1713786712
198,198,barcelos,41.5388,-8.6151,22.40,34,0,1.84,PT,1713786758
266,266,batabano,22.7164,-82.2881,23.92,70,0,2.25,CU,1713786823
386,386,camapua,-19.5314,-54.0439,25.39,64,0,2.15,BR,1713786935
416,416,al fqih ben calah,32.5009,-6.6906,25.86,27,0,2.11,MA,1713786964
525,525,karratha,-20.7377,116.8463,26.23,57,0,3.73,AU,1713787064


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,sabang,ID,5.8933,95.3214,68,
1,olonkinbyen,SJ,70.9221,-8.7187,66,
2,albany,US,42.6001,-73.9662,57,
3,hermanus,ZA,-34.4187,19.2345,70,
4,port mathurin,MU,-19.6833,63.4167,78,
...,...,...,...,...,...,...
551,elista,RU,46.3078,44.2558,25,
552,bubaque,GW,11.2833,-15.8333,70,
553,bereznik,RU,62.8551,42.7072,85,
554,caras,PE,-9.0467,-77.8114,82,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters for the API request
radius = 10000
params = {
    "limit": 1,  # Limit the number of results to 1 (i.e., find the nearest hotel)
    "categories": "accommodation.hotel",  # Specify the category to filter results (accommodation)
    "apiKey": geoapify_key  # Include the API key
}



# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row["Lat"]  # Replace with the actual column name containing latitude
    lon = row["Lng"]  # Replace with the actual column name containing longitude
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
sabang - nearest hotel: Sabang Hill
olonkinbyen - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
hermanus - nearest hotel: Aloe guest house
port mathurin - nearest hotel: Escale Vacances
udachny - nearest hotel: Вилюй
sorong - nearest hotel: Marina Mamberamo
adamstown - nearest hotel: No hotel found
port-aux-francais - nearest hotel: Keravel
bredasdorp - nearest hotel: Victoria Hotel
morondava - nearest hotel: Hotel Sharon Menabe
tame - nearest hotel: hotel libertadores
hamilton - nearest hotel: North Vista Manor
tazovsky - nearest hotel: Тазовчанка
whitehorse - nearest hotel: Town & Mountain Hotel
smithers - nearest hotel: Sunshine Inn Hotel
stanley - nearest hotel: Hotel 52
flin flon - nearest hotel: Copperbelt Hotel
blackmans bay - nearest hotel: Villa Howden
fort riley north - nearest hotel: Acorns Resort
lihue - nearest hotel: Kauai Palms
enewetak - nearest hotel: No hotel found
tsiombe - nearest hotel: No hotel found
lompoc - nearest ho

,City,Country,Lat,Lng,Humidity,Hotel Name
0,sabang,ID,5.8933,95.3214,68,Sabang Hill
1,olonkinbyen,SJ,70.9221,-8.7187,66,No hotel found
2,albany,US,42.6001,-73.9662,57,No hotel found
3,hermanus,ZA,-34.4187,19.2345,70,Aloe guest house
4,port mathurin,MU,-19.6833,63.4167,78,Escale Vacances
...,...,...,...,...,...,...
551,elista,RU,46.3078,44.2558,25,Островок
552,bubaque,GW,11.2833,-15.8333,70,Chez Julio
553,bereznik,RU,62.8551,42.7072,85,Отель Прибрежный
554,caras,PE,-9.0467,-77.8114,82,Hotel Luchita


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:

# Assuming you have latitude and longitude information for the map center
map_center = [hotel_df['Lat'].mean(), hotel_df['Lng'].mean()]  

# Create a DataFrame with necessary information for plotting
plot_df = hotel_df.copy()  # Assuming hotel_df is your DataFrame

# Create custom hover information as a list of strings
hover_info = plot_df.apply(lambda row: f"City: {row['City']}<br>Hotel: {row['Hotel Name']}<br>Country: {row['Country']}<br>Humidity: {row['Humidity']}", axis=1).tolist()

# Create a map plot with custom hover information
map_plot = plot_df.hvplot.points(x='Lng', y='Lat', geo=True, tiles='OSM', size=10, line_color='black', 
                                 hover_cols=['City','Humidity','Hotel Name', 'Country'], hover=hover_info,
                                color='City')

# Display the map plot
map_plot



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)